# NBA Sports Betting Analysis: How accurate are betting odds for NBA games?
GitHub Repo: https://github.com/ihern/418-Project.git 

UIC CS 418, Spring 2024

Group 12: 
- **Ronak Chaudhuri**, rchau5@uic.edu, RonakChaudhuri
- **Isaias Hernandez**, iherna50@uic.edu, ihern
- **Kevin Pereira**, kperei2@uic.edu, kperei2
- **Brandon Rosero**, broser2@uic.edu, brandon-rosero
- **Muhammad Sharif**, mhari8@uic.edu, msharitenn 

# Introduction
Changes Since Proposal: None

### Data

# Exploratory Data Analysis

# Visualizations


# Machine Learning Analysis 

# Reflection

# Next Step